In [2]:
from datetime import datetime
from functools import partial
import glob
from multiprocessing import Pool

# Disable multiprocesing for numpy/opencv. We already multiprocess ourselves, this would mean every subprocess produces
# even more threads which would lead to a lot of context switching, slowing things down a lot.
import os
os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
import scipy
import scipy.ndimage
import scipy.special
from scipy.spatial.transform import Rotation as R

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix

In [8]:
!ln -s /media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_images images
!ln -s /media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_maps maps
!ln -s /media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_lidar lidar

In [9]:
level5data = LyftDataset(data_path='.', json_path='../../input/3d-object-detection-for-autonomous-vehicles/train_data', verbose=True)

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 7.5 seconds.
Reverse indexing ...
Done reverse indexing in 2.1 seconds.


In [12]:
level5data.list_scenes()

host-a007-lidar0-1230485630199365106-12304856550990... [19-01-02 17:33:50]   25s, Palo Alto, #anns:1559
host-a007-lidar0-1230672860198383106-12306728850991... [19-01-04 21:34:20]   25s, Palo Alto, #anns:2969
host-a007-lidar0-1230936221299185986-12309362461986... [19-01-07 22:43:41]   25s, Palo Alto, #anns:683
host-a007-lidar0-1231093036199514746-12310930610996... [19-01-09 18:17:16]   25s, Palo Alto, #anns:3117
host-a009-lidar0-1231200854198312986-12312008790984... [19-01-11 00:14:14]   25s, Palo Alto, #anns:681
host-a005-lidar0-1231201437298603426-12312014621988... [19-01-11 00:23:57]   25s, Palo Alto, #anns:3064
host-a011-lidar0-1232401360198078026-12324013850983... [19-01-24 21:42:40]   25s, Palo Alto, #anns:3842
host-a011-lidar0-1232412236198491106-12324122610982... [19-01-25 00:43:56]   25s, Palo Alto, #anns:372
host-a007-lidar0-1232470052198454586-12324700770988... [19-01-25 16:47:32]   25s, Palo Alto, #anns:1614
host-a011-lidar0-1232485958298280666-12324859832000... [19-01-25 21

In [14]:
records = [(level5data.get('sample', record['first_sample_token'])['timestamp'], record) for record in level5data.scene]

In [15]:
level5data.scene

[{'log_token': 'da4ed9e02f64c544f4f1f10c6738216dcb0e6b0d50952e158e5589854af9f100',
  'first_sample_token': '24b0962e44420e6322de3f25d9e4e5cc3c7a348ec00bfa69db21517e4ca92cc8',
  'name': 'host-a101-lidar0-1241893239199111666-1241893264098084346',
  'description': '',
  'last_sample_token': '2346756c83f6ae8c4d1adec62b4d0d31b62116d2e1819e96e9512667d15e7cec',
  'nbr_samples': 126,
  'token': 'da4ed9e02f64c544f4f1f10c6738216dcb0e6b0d50952e158e5589854af9f100'},
 {'description': '',
  'first_sample_token': '3b30673b9d944ec6058ef5a8debb4c0a6fe075bca7076e06cf42a2bc10dc446e',
  'log_token': '0a6839d6ee6804113bb5591ed99cc70ad883d0cff396e3aec5e76e718771b30e',
  'name': 'host-a006-lidar0-1236037883198113706-1236037908098879296',
  'token': '0a6839d6ee6804113bb5591ed99cc70ad883d0cff396e3aec5e76e718771b30e',
  'last_sample_token': '5ec01e4634ca91751311eaafb45a9196ba8616bf05edc85593aff158db653a34',
  'nbr_samples': 126},
 {'first_sample_token': 'fcad797faa988f50be306e5fb6971de295602ee9116bd9ce05cfb514f

In [16]:
records

[(1557858039302414.8,
  {'log_token': 'da4ed9e02f64c544f4f1f10c6738216dcb0e6b0d50952e158e5589854af9f100',
   'first_sample_token': '24b0962e44420e6322de3f25d9e4e5cc3c7a348ec00bfa69db21517e4ca92cc8',
   'name': 'host-a101-lidar0-1241893239199111666-1241893264098084346',
   'description': '',
   'last_sample_token': '2346756c83f6ae8c4d1adec62b4d0d31b62116d2e1819e96e9512667d15e7cec',
   'nbr_samples': 126,
   'token': 'da4ed9e02f64c544f4f1f10c6738216dcb0e6b0d50952e158e5589854af9f100'}),
 (1552002683300887.5,
  {'description': '',
   'first_sample_token': '3b30673b9d944ec6058ef5a8debb4c0a6fe075bca7076e06cf42a2bc10dc446e',
   'log_token': '0a6839d6ee6804113bb5591ed99cc70ad883d0cff396e3aec5e76e718771b30e',
   'name': 'host-a006-lidar0-1236037883198113706-1236037908098879296',
   'token': '0a6839d6ee6804113bb5591ed99cc70ad883d0cff396e3aec5e76e718771b30e',
   'last_sample_token': '5ec01e4634ca91751311eaafb45a9196ba8616bf05edc85593aff158db653a34',
   'nbr_samples': 126}),
 (1551915097302078.5,


In [19]:
train_scenes = []
val_scenes = []
all_scenes = []

for sample in level5data.sample:
    
    print(sample["scene_token"])
    all_scenes.append(sample["scene_token"])

...


...


...


...


...


...


In [21]:
all_scenes = list(set(all_scenes))

In [23]:
import random
SEED = 42

random.seed(SEED)
random.shuffle(all_scenes)

In [24]:
train_scenes = all_scenes[:int(5 * len(all_scenes) / 6)]
val_scenes = all_scenes[int(5 * len(all_scenes) / 6):]

In [25]:
train_scenes

['328ee9d180f02a82516e8c5b4113ed18d8a36d2023c3ed3d5c219c1d7bb2cc79',
 '2983e04fcfd25de0207bbb706d6bfebbdd7b63b0844244c51203d85ba249c67d',
 '29bdd72da61e55ada4336dfaf15c98e8a0b8b7879c0746548f9ef2487915d5b1',
 '1793886dd3dbc6714dbb66e0167226edcb4fbe5f51de2b92d1918f925d8937ee',
 'f8e199f8b4b788e1c641b8b7fe3a693b3a5c807194ab4fd7672acd5c1f27e83d',
 '094c956927749d576052c7473a190ef86dc07e8ad7038b3132307518fa36af86',
 'ced54e296d660872200e6b29577e1d1f83f35fe40a63dcbad04135049175a6c1',
 '77e66f727e1e41bc239c5a479b3e08f46128a2ae2ec2623cedd0d934542638a3',
 '61e11c65542360b410222b55896bd6772285596d3eac3061866fe238cfa7dc28',
 '2bd00da3465bac6fdfe2177a12118650679e84e3133b2da8e4678d9d8d21b75d',
 'de4d664bac1b11cecd0d8fb43a226366da81d42a90d204f75bc3b4dc7d11709a',
 'af749b4da1ab470bd4af7bab1a1e9ea358349839844de30291ec3c1c15f94f48',
 '8602e56b989742b61894014842a805c83f0adc60bf16a53d74599df79e1191d4',
 '473093b48a7cb78d05e36245fd2dbd12d66ded7dab1ecb862945390b8a765c0a',
 'b45f513a29561cc533d5a975a1bcf3ab

In [27]:
with open("train.txt", "w") as f:
    for item in train_scenes:
        f.write("%s\n" % item)

In [30]:
with open("train.txt") as f:
    lines = f.read().splitlines()